In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Define your csv_columns list
csv_columns = ["Business Activity Description", "Business Activity Vendor", "Business Activity Cost USD", "Business Activity Comment", "2017 NAICS Title"]

file_folder = '/content/drive/My Drive/224_project/'

# Specify Google Drive folder ID and file names
folder_id = "1FwqVD1UvjaoSC8UksUBhVkPzPQy95iTg"

total_file_name = 'sustainability_business_activities.csv'
train_file_name = 'sustainability_business_activities_training.csv'
test_file_name = 'sustainability_business_activities_test.csv'


In [ ]:
import pandas as pd
from io import StringIO

def find_file_id_by_name(drive, folder_id, file_name):
    """Search for a file by name in the specified Google Drive folder."""
    query = f"'{folder_id}' in parents and trashed=false and title='{file_name}'"
    file_list = drive.ListFile({'q': query}).GetList()
    return file_list[0]['id'] if file_list else None

# Assuming `gauth` and `drive` are already initialized
# Your DataFrames: train_dataset and test_dataset

def upload_dataframe(drive, folder_id, df, file_name):
    # Convert DataFrame to CSV string
    csv_string = df.to_csv(index=False)
    content = StringIO(csv_string)

    # Check if file exists
    file_id = find_file_id_by_name(drive, folder_id, file_name)

    if file_id:
        # File exists, update the content
        file = drive.CreateFile({'id': file_id})
        file.SetContentString(csv_string)
        file.Upload()
        print(f"Updated: {file_name}")
    else:
        # File doesn't exist, create a new one
        file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}]})
        file.SetContentString(csv_string)
        file.Upload()
        print(f"Created: {file_name}")

In [ ]:
'''
import pandas as pd
import numpy as np
import random

# Helper functions
def introduce_minor_errors(text):
    """Introduce minor spelling mistakes in the text."""
    if random.random() < 0.15:  # 15% chance
      errors_introduced = 0
      max_errors = random.randint(2, 3)  # Decide to introduce 2 or 3 minor errors

      while errors_introduced < max_errors and len(text) > 4:  # Ensure text is long enough to alter
          error_type = random.choice(['substitute', 'omit', 'swap'])
          error_index = random.randint(1, len(text) - 2)  # Avoid beginning and end of the text for simplicity

          if error_type == 'substitute':
              # Substitute a character with a nearby character (mimicking common typing errors)
              substitutions = {'a': 's', 's': 'a', 'd': 'f', 'i': 'o', 'o': 'p', 'e': 'r', 'r': 't'}
              if text[error_index] in substitutions:
                  text = text[:error_index] + substitutions[text[error_index]] + text[error_index + 1:]
                  errors_introduced += 1

          elif error_type == 'omit':
              # Omit a character
              text = text[:error_index] + text[error_index + 1:]
              errors_introduced += 1

          elif error_type == 'swap':
              # Swap two adjacent characters
              if error_index < len(text) - 1:  # Ensure there's a character to swap with
                  text = text[:error_index] + text[error_index + 1] + text[error_index] + text[error_index + 2:]
                  errors_introduced += 1
    return text

def introduce_major_errors(text):
    """Replace or scramble parts of the text to introduce major errors."""
    if random.random() < 0.15:  # 15% chance
        # Randomly choose between scrambling or inserting irrelevant text
        if random.random() < 0.5:
            return ''.join(random.sample(text, len(text)))
        else:
            return "Irrelevant text " + ''.join(random.sample(text, len(text)))
    return text
'''
print('#Commented Code, not using anymore')

#Commented Code, not using anymore


In [ ]:
# Simplistic data generation. No support for generating error. Vendor and
# comments are NOT activity specific. As a result LLM can't learn from
# Vendor and Comment fields
'''
import pandas as pd
import numpy as np
import random

# NAICS Titles
naics_titles = [
    "Soybean Farming", "Oilseed (except Soybean) Farming", "Dry Pea and Bean Farming", "Wheat Farming",
    "Corn Farming", "Rice Farming", "Oilseed and Grain Combination Farming", "All Other Grain Farming",
    "Potato Farming", "Other Vegetable (except Potato) and Melon Farming", "Orange Groves", "Citrus (except Orange) Groves",
    "Apple Orchards", "Grape Vineyards", "Strawberry Farming", "Berry (except Strawberry) Farming",
    "Tree Nut Farming", "Fruit and Tree Nut Combination Farming", "Other Noncitrus Fruit Farming",
    "Mushroom Production", "Other Food Crops Grown Under Cover", "Nursery and Tree Production",
    "Floriculture Production", "Tobacco Farming", "Cotton Farming", "Sugarcane Farming", "Hay Farming",
    "Sugar Beet Farming", "Peanut Farming", "All Other Miscellaneous Crop Farming",
    "Beef Cattle Ranching and Farming", "Cattle Feedlots", "Dairy Cattle and Milk Production",
    "Dual-Purpose Cattle Ranching and Farming", "Hog and Pig Farming", "Chicken Egg Production",
    "Broilers and Other Meat Type Chicken Production", "Turkey Production", "Poultry Hatcheries",
    "Other Poultry Production", "Sheep Farming", "Goat Farming", "Finfish Farming and Fish Hatcheries",
    "Shellfish Farming", "Other Aquaculture", "Apiculture", "Horses and Other Equine Production",
    "Fur-Bearing Animal and Rabbit Production", "All Other Animal Production"
]

# Sample vendors for simulation
vendors = ["AgriCorp", "BioHarvest", "CropGen", "EcoFarms", "FreshFields"]

# Function to generate a random cost
def generate_cost():
    return round(random.uniform(1000, 50000), 2)

# Function to generate a cohesive comment
def generate_comment(naics_title):
    return f"Procured for {naics_title.lower()} operations."

# Generate synthetic data
def generate_data(naics_titles, num_records=5000):
    data = []
    for _ in range(num_records):
        naics_title = random.choice(naics_titles)
        data.append({
            "business_activity_description": f"Activities related to {naics_title.lower()}.",
            "Cost": generate_cost(),
            "Vendor": random.choice(vendors),
            "Comment": generate_comment(naics_title),
            "2017 NAICS Title": naics_title
        })
    return pd.DataFrame(data)

# Generate the datasets
data = generate_data(naics_titles, 6000)

# Split into training and test sets
train_data = data.sample(frac=0.833, random_state=42)  # 5000 records for training
test_data = data.drop(train_data.index)  # Remaining 1000 records for testing

train_data.to_csv('business_activities_training_data_enhanced.csv', index=False)
test_data.to_csv('business_activities_test_data_enhanced.csv', index=False)
'''
print('#Commented Code, not using anymore')

#Commented Code, not using anymore


In [ ]:
# Bit more advanced data generation. Support for generating error
# in the description field. Vendor and comments are NOT activity specific.
# As a result LLM can't learn from Vendor and Comment fields

'''
import pandas as pd
import numpy as np
import random

def generate_data(naics_titles, num_records=5000):
    data = []
    for _ in range(num_records):
        naics_title = random.choice(naics_titles)
        description = f"Activities related to {naics_title.lower()}."
        description_with_errors = description
        # Introduce errors
        if random.random() < 0.5:
          description_with_errors = introduce_minor_errors(description)
        else:
          description_with_errors = introduce_major_errors(description_with_errors)

        data.append({
            "business_activity_description": description_with_errors,
            "Cost": generate_cost(),
            "Vendor": random.choice(vendors),
            "Comment": generate_comment(naics_title),
            "2017 NAICS Title": naics_title
        })
    return pd.DataFrame(data)

# Generate the datasets with errors introduced
data_with_errors = generate_data(naics_titles, 6000)

# Split into training and test sets
train_data_with_errors = data_with_errors.sample(frac=0.833, random_state=42)  # 5000 records for training
test_data_with_errors = data_with_errors.drop(train_data_with_errors.index)  # Remaining 1000 records for testing

# Save to CSV files
train_data_with_errors.to_csv('business_activities_training_data_with_errors.csv', index=False)
test_data_with_errors.to_csv('business_activities_test_data_with_errors.csv', index=False)
'''
print('#Commented Code, not using anymore')

#Commented Code, not using anymore


In [ ]:
# Define NAICS Titles and their associated business activities, vendors, and comments
naics_info = {
    "Support Activities for Metal Mining": {
        "activities": {
            "Drilling services for zinc exploration": {
                "vendors": ["ZincDrill Corp", "MetalExplorer Inc.", "DrillZinc Solutions", "ZincQuest Services", "ExploreZinc Co."],
                "comments": [
                    "Precision drilling for accurate zinc location.",
                    "Advanced geological analysis ensured.",
                    "Minimized environmental footprint during exploration."
                ]
            },
            "Environmental impact assessment for mining": {
                "vendors": ["EcoImpact Assessors", "GreenMine Consultants", "SustainAssess Ltd.", "EcoMine Services", "EnviroImpact Analytics"],
                "comments": [
                    "Comprehensive ecosystem evaluation conducted.",
                    "Strategies for water conservation implemented.",
                    "Biodiversity preservation plans developed."
                ]
            },
            "Ore transportation logistics": {
                "vendors": ["HeavyLoad Transport", "OreMovers Inc.", "MetalFreight Co.", "RockShip Logistics", "MineCart Express"],
                "comments": [
                    "Efficient ore transport solutions.",
                    "State-of-the-art logistics management.",
                    "Eco-friendly transportation methods."
                ]
            },
            "Mine site reclamation services": {
                "vendors": ["EcoReclaim Ltd.", "LandRestore Co.", "ReGreening Services", "TerraHeal Solutions", "NatureMend Tech"],
                "comments": [
                    "Restoration of mining sites to natural state.",
                    "Promoting biodiversity in reclaimed areas.",
                    "Innovative land rehabilitation techniques."
                ]
            },
            "Mineral processing consultancy": {
                "vendors": ["MineralTech Advisors", "OreRefine Partners", "ProcessMax Consultants", "ExtractWell Analytics", "BeneficiatePro Services"],
                "comments": [
                    "Optimization of mineral processing operations.",
                    "Implementation of cost-saving techniques.",
                    "Maximizing ore recovery rates."
                ]
            },
            "Mining equipment maintenance": {
                "vendors": ["MineGear Service Co.", "DigTech Repair Solutions", "EquipFix Services", "HeavyDuty Maintenance", "DrillAndGrind Engineers"],
                "comments": [
                    "Ensuring operational efficiency and safety.",
                    "Rapid response for equipment repairs.",
                    "Comprehensive maintenance plans."
                ]
            },
            "Safety training for mine workers": {
                "vendors": ["SafeMine Training Academy", "MinerGuard Education Services", "SafetyFirst Workshops", "ProtectWell Training Programs", "HazardAware Courses"],
                "comments": [
                    "Empowering workers with safety knowledge.",
                    "Compliance with mining safety regulations.",
                    "Advanced risk management training."
                ]
            },
            "Environmental monitoring services": {
                "vendors": ["EcoWatch Services", "EnviroSense Monitoring", "GreenScan Analytics", "BioTrack Environmental", "NatureGuard Surveillance"],
                "comments": [
                    "Continuous environmental impact assessment.",
                    "Utilizing advanced monitoring technologies.",
                    "Data-driven environmental protection strategies."
                ]
            },
            "Heavy machinery leasing": {
                "vendors": ["MegaMach Leasing Co.", "HeavyLift Rental Services", "IronHorse Machinery", "PowerPlant Equipment Leasing", "ToughTrack Leases"],
                "comments": [
                    "Providing flexible leasing options.",
                    "Access to latest mining machinery models.",
                    "Cost-effective solutions for equipment needs."
                ]
            },
            "Rock blasting and removal": {
                "vendors": ["BlastAway Co.", "RockClear Services", "ExploTech Blasting", "RubbleManage Inc.", "DetonateRock Solutions"],
                "comments": [
                    "Precision blasting for site preparation.",
                    "Safe removal of blasted rock material.",
                    "Minimizing vibrations and environmental impact."
                ]
            }
        }
    },
    "Sewage Treatment Facilities": {
        "activities": {
            "Wastewater treatment process optimization": {
                "vendors": ["WaterClean Solutions", "PureFlow Tech", "EcoWater Systems", "AquaPurify Innovations", "CleanStream Technologies"],
                "comments": [
                    "Enhanced filtration techniques applied.",
                    "Energy-efficient treatment methods introduced.",
                    "Significant reduction in chemical usage achieved."
                ]
            },
            "Sludge dewatering and disposal": {
                "vendors": ["SludgeAway Technologies", "DryMatter Solutions", "EcoSludge Services", "Solidify Systems", "WasteNo Sludge Co."],
                "comments": [
                    "Advanced centrifugation for moisture reduction.",
                    "Eco-friendly disposal methods utilized.",
                    "Conversion of sludge to renewable energy explored."
                ]
            },
            "Chemical treatment of wastewater": {
                "vendors": ["ChemClean Water Solutions", "AquaChem Tech", "PureTreat Chemicals", "WasteNeutralize Enterprises", "EcoChem Services"],
                "comments": [
                    "Advanced chemical treatments for purification.",
                    "Reduction of harmful pathogens in effluent.",
                    "Eco-friendly chemical usage policies."
                ]
            },
            "Maintenance of treatment plant infrastructure": {
                "vendors": ["AquaMaintain Ltd.", "PlantCare Engineering", "FacilityFix Services", "WaterWorks Maintenance", "SewageSustain Co."],
                "comments": [
                    "Regular inspection and maintenance of facilities.",
                    "Preventative maintenance to avoid downtime.",
                    "Upgrading infrastructure for efficiency improvements."
                ]
            },
            "Odor control in sewage facilities": {
                "vendors": ["ScentGuard Technologies", "OdorShield Solutions", "FreshAir Environmental", "SmellBusters Co.", "AromaControl Systems"],
                "comments": [
                    "Implementing natural odor control measures.",
                    "Advanced filtration systems to reduce smells.",
                    "Improving community relations with odor management."
                ]
            },
            "Grease and fat removal services": {
                "vendors": ["GreaseGone Solutions", "FatFree Services", "ClearGrease Co.", "LipidLifters Inc.", "EcoFatClean Tech"],
                "comments": [
                    "Preventing blockages in sewage systems.",
                    "Eco-friendly disposal of grease and fats.",
                    "Maintaining cleanliness and flow efficiency."
                ]
            },
            "Renewable energy generation from sewage": {
                "vendors": ["BioEnergy Water Solutions", "RenewFlow Tech", "AquaPower Renewable", "WasteToWatt Co.", "EcoVolt Energy"],
                "comments": [
                    "Harnessing biogas from sewage treatment.",
                    "Contributing to sustainability goals.",
                    "Innovative energy recovery techniques."
                ]
            },
            "Sewage system digital monitoring": {
                "vendors": ["FlowSense Monitoring", "AquaTrack Technologies", "DigitalWater Watch", "EcoMonitor Solutions", "Waterwise Surveillance"],
                "comments": [
                    "Real-time data on sewage treatment operations.",
                    "Predictive maintenance with digital tools.",
                    "Enhancing system efficiency through technology."
                ]
            },
            "Public health impact assessments": {
                "vendors": ["HealthWater Consultancy", "PublicSafe Water Services", "EcoHealth Assessments", "WaterWell Public Health", "AquaHealth Analytics"],
                "comments": [
                    "Assessing the impact of sewage treatment on public health.",
                    "Strategies for mitigating health risks.",
                    "Collaboration with public health authorities."
                ]
            },
            "Stormwater runoff management": {
                "vendors": ["StormSafe Solutions", "RunoffGuard Co.", "RainFlow Management", "EcoStorm Services", "WaterRunoff Solutions"],
                "comments": [
                    "Innovative solutions for stormwater treatment.",
                    "Reducing pollution from runoff.",
                    "Improving water quality in local water bodies."
                ]
            }
        }
    },
    "New Single-Family Housing Construction (except For-Sale Builders)": {
        "activities": {
            "Architectural design for family homes": {
                "vendors": ["DesignBuild Architects", "EcoHomes Design Co.", "FutureNest Architecture", "GreenBlueprints Studio", "HarmonyDwell Designs"],
                "comments": [
                    "Custom designs to fit client lifestyles.",
                    "Incorporation of green building practices.",
                    "Focus on energy efficiency and sustainability."
                ]
            },
            "Foundation and framing services": {
                "vendors": ["SolidBase Constructors", "FrameRight Solutions", "EcoFoundation Systems", "SecureFrame Builders", "GroundUp Framing Co."],
                "comments": [
                    "Utilizing durable and sustainable materials.",
                    "Advanced techniques for structural integrity.",
                    "Timely completion of foundational and framing stages."
                ]
            },
            "Plumbing installation and fixtures": {
                "vendors": ["FlowTech Plumbing", "AquaSafe Installations", "EcoWater Plumbing Co.", "PureFlow Systems", "Streamline Plumbing Solutions"],
                "comments": [
                    "High-quality fixtures to reduce water usage.",
                    "Innovative solutions for water heating and recycling.",
                    "Reliable installation ensuring long-term service."
                ]
            },
            "Electrical wiring and smart home integration": {
                "vendors": ["BrightFuture Electric", "SmartWatt Solutions", "EcoElectrics Co.", "PowerSafe Wiring", "InnovateHome Systems"],
                "comments": [
                    "Energy-efficient lighting and appliances.",
                    "Integration of smart home technologies.",
                    "Focus on safety and energy savings."
                ]
            },
            "Roofing and insulation services": {
                "vendors": ["EcoRoof Systems", "SecureCover Roofing", "WarmthWrap Insulation", "ShieldTop Roofs", "InsulateRight Services"],
                "comments": [
                    "Sustainable roofing materials for durability.",
                    "Advanced insulation for climate control.",
                    "Options for solar panel installation."
                ]
            },
            "Landscaping and outdoor living spaces": {
                "vendors": ["GreenEscape Landscaping", "OutdoorOasis Designs", "NatureNest Landscape Co.", "EcoYard Landscaping", "LivingSpaces Outdoors"],
                "comments": [
                    "Designs that complement natural surroundings.",
                    "Creating functional and beautiful outdoor spaces.",
                    "Incorporation of native plants and eco-friendly practices."
                ]
            },
            "Interior finishing and painting": {
                "vendors": ["InsideStyle Finishers", "ColorCraft Painters", "EcoInterior Coatings", "FinishLine Interiors", "BrightWalls Solutions"],
                "comments": [
                    "Use of low-VOC paints for indoor air quality.",
                    "Attention to detail in finishing work.",
                    "Custom interior designs to match homeowner preferences."
                ]
            },
            "Flooring installation and treatments": {
                "vendors": ["EcoFloors Installations", "GroundArt Flooring", "NatureWalk Surfaces", "StepWell Flooring Co.", "DuraFloor Solutions"],
                "comments": [
                    "Eco-friendly flooring options available.",
                    "Durable treatments for high-traffic areas.",
                    "Wide range of materials and finishes."
                ]
            },
            "Window and door installation": {
                "vendors": ["ClearView Windows", "SecureEntrance Doors", "EcoPane Solutions", "BrightAccess Installations", "OpenWay Doors & Windows"],
                "comments": [
                    "Energy-efficient windows for thermal control.",
                    "Secure and durable doors for safety.",
                    "Custom designs to enhance aesthetic appeal."
                ]
            },
            "Heating, Ventilation, and Air Conditioning (HVAC) systems": {
                "vendors": ["ClimateControl HVAC Co.", "EcoVent Systems", "PureAir Solutions", "ComfortZone HVAC", "ThermalTech Installations"],
                "comments": [
                    "High-efficiency systems for energy savings.",
                    "Advanced air filtration for health and comfort.",
                    "Smart systems for climate control and monitoring."
                ]
            }
        }
    },
    "Timber Tract Operations": {
        "activities": {
            "Tree planting and reforestation": {
                "vendors": ["GreenSeed Reforestation", "EcoGrow Forestry", "TreeRevive Operations", "ForestRenew Planters", "SaplingStart Co."],
                "comments": [
                    "Ensuring sustainability of timber resources.",
                    "Specializing in native species to promote biodiversity.",
                    "Utilizing GPS mapping for optimal planting strategies."
                ]
            },
            "Timber harvesting operations": {
                "vendors": ["LogCutters Inc.", "TimberYield Harvesting", "WoodFell Solutions", "ForestExtract Co.", "ClearCut Logging"],
                "comments": [
                    "Precision cutting to minimize waste.",
                    "Adhering to sustainable harvest practices.",
                    "Implementing low-impact logging techniques."
                ]
            },
            "Forest management and conservation": {
                "vendors": ["EcoForest Management", "GreenCanopy Consultants", "BioDiverse Forestry", "ConservForest Services", "SustainWood Group"],
                "comments": [
                    "Balancing timber production with ecosystem health.",
                    "Developing long-term forest management plans.",
                    "Monitoring health and growth of forest tracts."
                ]
            },
            "Pest and disease control": {
                "vendors": ["PestAway TimberCare", "TreeGuardians Co.", "ForestShield Services", "EcoPest Forestry", "TimberHealth Solutions"],
                "comments": [
                    "Employing environmentally friendly pest control methods.",
                    "Regular monitoring for early disease detection.",
                    "Implementing integrated pest management (IPM) strategies."
                ]
            },
            "Timber valuation and appraisal": {
                "vendors": ["WoodWorth Appraisals", "TimberValue Consultants", "EstateLogs Valuation", "TreeEquity Services", "LumberGrade Assessors"],
                "comments": [
                    "Accurate valuation for sales and acquisitions.",
                    "Expert appraisals based on market trends.",
                    "Providing comprehensive forestry investment analysis."
                ]
            },
            "Road construction and maintenance for logging": {
                "vendors": ["LogPath Engineers", "ForestRoads Construction", "TimberTrail Co.", "EcoAccess Routes", "WoodWay Builders"],
                "comments": [
                    "Building access roads with minimal environmental impact.",
                    "Maintaining roads for safety and sustainability.",
                    "Utilizing erosion control measures in road design."
                ]
            },
            "Timber marketing and sales": {
                "vendors": ["WoodMarket Solutions", "LogSales Network", "TimberTrade Co.", "ForestProducts Exchange", "EcoTimber Sales"],
                "comments": [
                    "Connecting sellers with global markets.",
                    "Promoting sustainable timber products.",
                    "Implementing traceability for timber origin."
                ]
            },
            "Wildlife habitat management": {
                "vendors": ["WildHabitat Forestry", "EcoFauna Management", "BioHaven Wildlife Services", "NatureBalance Co.", "HabitatHarmony Solutions"],
                "comments": [
                    "Integrating wildlife conservation into timber operations.",
                    "Creating buffer zones to protect sensitive species.",
                    "Monitoring biodiversity as a key management metric."
                ]
            },
            "Water resource management on timber lands": {
                "vendors": ["AquaForest Management", "StreamGuard Services", "EcoWatershed Co.", "TimberSpring WaterCare", "ForestRivers Solutions"],
                "comments": [
                    "Protecting water quality through best management practices.",
                    "Restoring stream and river habitats affected by logging.",
                    "Implementing rainwater harvesting for drought resilience."
                ]
            },
            "Fire prevention and control measures": {
                "vendors": ["FlameWard Forestry", "FireBreak Solutions", "EcoFireGuard Co.", "BlazeControl Services", "WildfirePrevent Consultants"],
                "comments": [
                    "Strategic creation of firebreaks and buffer zones.",
                    "Utilizing controlled burns for underbrush management.",
                    "Deploying early detection systems for rapid response."
                ]
            }
        }
    },
    "Uranium-Radium-Vanadium Ore Mining": {
        "activities": {
            "Uranium ore extraction": {
                "vendors": ["UraniumExtract Ltd.", "EcoUranium Solutions", "PureUranium Co.", "RadMine Operations", "VanadoUranium Group"],
                "comments": [
                    "High-grade uranium extraction with minimal environmental impact.",
                    "Implementing advanced safety measures for workers.",
                    "Utilizing innovative technologies for efficient ore processing."
                ]
            },
            "Radium isolation processes": {
                "vendors": ["RadiumTech Innovations", "EcoRad Solutions", "RadPurity Extractors", "IsolateRadium Co.", "RadiantRadium Services"],
                "comments": [
                    "Specialized techniques for radium isolation from ore.",
                    "Ensuring environmental safety in radium processing.",
                    "Conducting thorough radiation safety assessments."
                ]
            },
            "Vanadium mining and processing": {
                "vendors": ["VanadiumValley Co.", "EcoVan Mining", "VanaTech Processing", "PureVan Extracts", "InnoVanadium Solutions"],
                "comments": [
                    "Extracting vanadium with high purity levels.",
                    "Focusing on sustainable mining practices.",
                    "Applying state-of-the-art processing technology."
                ]
            },
            "Ore crushing and milling": {
                "vendors": ["CrushMill Ore Services", "EcoCrush Technologies", "MegaMillers Ltd.", "OreGrind Solutions", "FineCrush Co."],
                "comments": [
                    "Efficient ore crushing for optimal extraction.",
                    "Reducing energy consumption in milling operations.",
                    "Innovating in dust control techniques during processing."
                ]
            },
            "Leaching process for uranium extraction": {
                "vendors": ["LeachPro Uranium Services", "EcoLeach Solutions", "UraLeach Tech", "PureExtract Leaching", "LeachWell Uranium"],
                "comments": [
                    "Maximizing uranium yield through advanced leaching.",
                    "Minimizing environmental footprint of leaching operations.",
                    "Enhancing safety protocols in chemical leaching processes."
                ]
            },
            "Radiation safety and environmental monitoring": {
                "vendors": ["RadSafe Monitoring Co.", "EnviroRad Services", "SafeRad Technologies", "EcoMonitor Radiation", "RadGuard Analytics"],
                "comments": [
                    "Comprehensive radiation safety measures in place.",
                    "Continuous environmental monitoring for public safety.",
                    "Implementing best practices in radiation control."
                ]
            },
            "Tailings management and disposal": {
                "vendors": ["TailingsCare Solutions", "EcoTails Management", "SafeDisposal Systems", "TailingsTech Co.", "EnviroTailings Services"],
                "comments": [
                    "Innovative tailings management for sustainability.",
                    "Implementing secure disposal methods.",
                    "Focus on reducing tailings pond footprint."
                ]
            },
            "Water treatment in mining operations": {
                "vendors": ["AquaMine Solutions", "CleanWater Mining Co.", "EcoWater Treatment", "WaterPure Systems", "MineWaterTech"],
                "comments": [
                    "Advanced water treatment for mining effluents.",
                    "Ensuring water quality exceeds environmental standards.",
                    "Recycling and reusing water within mining operations."
                ]
            },
            "Transport and logistics for mined ore": {
                "vendors": ["OreTrans Logistics", "EcoTrans Mining Co.", "SafeCargo Solutions", "MineMovers Transport", "RadVan Logistics"],
                "comments": [
                    "Efficient transport solutions for uranium ore.",
                    "Implementing eco-friendly logistics practices.",
                    "Enhanced safety measures for radioactive material transport."
                ]
            },
            "Mine reclamation and ecosystem restoration": {
                "vendors": ["ReclaimEarth Services", "EcoRestore Mining", "GreenMine Reclamation", "RestoreLand Co.", "MineHeal Environmental"],
                "comments": [
                    "Restoring mined lands to their natural state.",
                    "Promoting biodiversity in mine reclamation projects.",
                    "Utilizing native plants in ecosystem restoration."
                ]
            }
        }
    },
    "Automobile Driving Schools": {
        "activities": {
            "Beginner driving lessons": {
                "vendors": ["StartRight Driving School", "FirstGear Lessons", "NewDriver Academy", "RoadBasics School", "DriveStart Educators"],
                "comments": [
                    "Tailored to new drivers for a solid foundation.",
                    "Emphasizes safety and road awareness from day one.",
                    "Instructors certified with a focus on patience and clarity."
                ]
            },
            "Advanced driving techniques": {
                "vendors": ["ProDrive Techniques", "AdvancedWheel School", "SkillShift Academy", "EliteDrivers Course", "PrecisionPilots Lessons"],
                "comments": [
                    "Covers defensive driving, evasive maneuvers, and more.",
                    "Designed for experienced drivers to enhance skills.",
                    "Utilizes simulation and on-road training for mastery."
                ]
            },
            "Defensive driving courses": {
                "vendors": ["SafeGuard Driving", "DefendDrive School", "ShieldOn Driving Academy", "FortressWheel Training", "DefensiveRoads School"],
                "comments": [
                    "Focuses on anticipating and avoiding road hazards.",
                    "Incorporates latest safety techniques and technologies.",
                    "Reduces risk of accidents through proactive training."
                ]
            },
            "Driving test preparation": {
                "vendors": ["TestReady Driving School", "PassFirst Go", "LicenseQuest Prep", "DriveTest Success Academy", "ExamGear Drivers"],
                "comments": [
                    "Comprehensive review and mock tests for confidence.",
                    "Highlights common test pitfalls and how to avoid them.",
                    "Personalized coaching based on student's weaknesses."
                ]
            },
            "Eco-driving lessons": {
                "vendors": ["EcoWheel Education", "GreenDrive School", "SustainaRide Lessons", "EcoPilot Academy", "CleanDrive Instructors"],
                "comments": [
                    "Teaches fuel-efficient driving habits.",
                    "Contributes to environmental protection and cost savings.",
                    "Incorporates hybrid and electric vehicle training."
                ]
            },
            "Vehicle maintenance workshops": {
                "vendors": ["AutoUpkeep Workshop", "MaintainMasters Class", "CarCare Clinic", "VehicleVitals Lessons", "TuneUp Teach"],
                "comments": [
                    "Hands-on learning for basic vehicle upkeep and emergency repairs.",
                    "Empowers drivers with knowledge to minimize maintenance costs.",
                    "Courses tailored for different vehicle types and models."
                ]
            },
            "Motorcycle driving lessons": {
                "vendors": ["TwoWheels Training Academy", "MotoLearn School", "BikeBasics Instructors", "CycleSafe Lessons", "RiderCraft Education"],
                "comments": [
                    "Specialized curriculum for aspiring motorcyclists.",
                    "Emphasizes balance, control, and road safety.",
                    "Includes gear selection and maintenance advice."
                ]
            },
            "Commercial driving license (CDL) training": {
                "vendors": ["CDLPro Academy", "HeavyHaulers School", "TruckMasters Training", "BigRig Educators", "CommercialDrivers Course"],
                "comments": [
                    "Prepares students for CDL exams with intensive training.",
                    "Covers commercial vehicle laws, log keeping, and cargo safety.",
                    "Includes behind-the-wheel practice with various truck types."
                ]
            },
            "Teen driving safety programs": {
                "vendors": ["YouthDrive Initiative", "TeenWheelers School", "SafeStart Young Drivers", "NextGen Drivers Academy", "GuardianRoad Teens"],
                "comments": [
                    "Designed to instill responsible driving habits early.",
                    "Interactive sessions with emphasis on peer influence.",
                    "Partners with schools and communities for outreach."
                ]
            },
            "Driver rehabilitation and retraining": {
                "vendors": ["DriveAgain Center", "WheelRecovery Services", "BackOnRoad Academy", "ReDrive Clinic", "SteerClear Program"],
                "comments": [
                    "Support for drivers recovering from injuries or disabilities.",
                    "Adapts teaching methods and vehicles to individual needs.",
                    "Focuses on restoring confidence and independence on the road."
                ]
            }
        }
    },
    "Offices of Dentists": {
        "activities": {
            "General dental check-ups": {
                "vendors": ["DentalCare Associates", "BrightSmile Clinics", "HealthyTeeth Dental", "OralWellness Providers", "SmileFirst Services"],
                "comments": [
                    "Emphasis on preventative care and oral hygiene.",
                    "Routine examinations to maintain dental health.",
                    "Utilizing the latest diagnostic technology for comprehensive check-ups."
                ]
            },
            "Cosmetic dentistry services": {
                "vendors": ["AestheticDental Arts", "CosmoSmile Dental Studio", "BrightenDent Aesthetics", "SmileDesign Experts", "GlamourSmiles Clinic"],
                "comments": [
                    "Specializing in smile makeovers and aesthetic improvements.",
                    "Offering a range of services from whitening to veneers.",
                    "Personalized cosmetic plans to enhance patients' smiles."
                ]
            },
            "Orthodontic treatments": {
                "vendors": ["AlignOrtho Care", "BraceBright Orthodontics", "StraightPath Dental", "HarmonyOrtho Solutions", "PerfectSmile Braces"],
                "comments": [
                    "Expertise in teeth alignment and bite correction.",
                    "Utilizing modern braces and clear aligner technologies.",
                    "Tailored orthodontic plans for all age groups."
                ]
            },
            "Pediatric dentistry": {
                "vendors": ["LittleSmiles Pediatric", "KidsDental Zone", "TinyTeeth Specialists", "YouthfulBites Dental", "ChildCare Dentistry"],
                "comments": [
                    "Dedicated care for children's dental health.",
                    "Creating a friendly and reassuring environment for young patients.",
                    "Focusing on preventive care and oral health education."
                ]
            },
            "Periodontal disease treatment": {
                "vendors": ["GumGuardians Clinic", "PerioProtect Services", "DeepClean Dental", "GumHealth Specialists", "BeneathTheGums Care"],
                "comments": [
                    "Advanced treatment options for gum disease.",
                    "Personalized care plans to halt disease progression.",
                    "Emphasis on minimally invasive procedures for gum health."
                ]
            },
            "Dental implant services": {
                "vendors": ["ImplantInnovations Dental", "AnchorDent Implants", "FoundationTeeth Solutions", "PermanentSmiles Clinic", "ToothRoot Systems"],
                "comments": [
                    "State-of-the-art dental implant solutions.",
                    "Restoring functionality with a natural look and feel.",
                    "Comprehensive care from consultation to post-operative follow-up."
                ]
            },
            "Emergency dental services": {
                "vendors": ["RapidRelief Dental Care", "EmergencyDent Assist", "24Hour Dental Clinic", "UrgentCare ToothSavers", "ImmediateHelp Dentists"],
                "comments": [
                    "Prompt care for dental emergencies and injuries.",
                    "Available around the clock for urgent dental needs.",
                    "Equipped to handle a wide range of dental emergencies."
                ]
            },
            "Dental hygiene services": {
                "vendors": ["CleanBite Hygiene", "FreshMouth Dental Care", "HygieneMasters Clinic", "PlaqueFighters Services", "BrightGleam Cleanings"],
                "comments": [
                    "Professional cleaning to prevent cavities and gum disease.",
                    "Educating patients on effective oral hygiene practices.",
                    "Utilizing gentle techniques for a comfortable experience."
                ]
            },
            "Endodontic (root canal) therapy": {
                "vendors": ["RootCare Specialists", "EndoHeal Dental", "PainFreeRoots Clinic", "CanalCure Endodontics", "InsideTooth Care"],
                "comments": [
                    "Expert care for root canal therapy and tooth pain relief.",
                    "Utilizing advanced techniques for successful outcomes.",
                    "Focus on patient comfort and preserving natural teeth."
                ]
            },
            "Dental X-ray and imaging services": {
                "vendors": ["ImageDent Diagnostics", "RadiantSmile Imaging", "XrayVision Dental", "OralScan Services", "DentoGraphix Clinic"],
                "comments": [
                    "High-definition imaging for accurate diagnostics.",
                    "Minimally invasive X-ray techniques for patient safety.",
                    "Comprehensive imaging services to support dental treatments."
                ]
            }
        }
    },
    "Kidney Dialysis Centers": {
        "activities": {
            "Hemodialysis treatment": {
                "vendors": ["CleanFilter Dialysis", "HemoCare Services", "PureBlood Solutions", "LifeStream Dialysis", "RenalTech Centers"],
                "comments": [
                    "State-of-the-art hemodialysis equipment for efficient blood filtering.",
                    "Personalized care plans tailored to individual patient needs.",
                    "Continuous monitoring and adjustment for optimal treatment outcomes."
                ]
            },
            "Peritoneal dialysis support": {
                "vendors": ["HomeDialysis Plus", "PeriCare Solutions", "InnerCleanse Therapy", "GentleDialysis Supplies", "StayPure Systems"],
                "comments": [
                    "Comprehensive support for at-home peritoneal dialysis patients.",
                    "Training and ongoing assistance for patients and families.",
                    "Supply of high-quality dialysis fluids and equipment."
                ]
            },
            "Kidney health education": {
                "vendors": ["RenalEd Partners", "KidneyWise Academy", "HealthStream Education", "NephroKnow Institute", "RenalAware Program"],
                "comments": [
                    "Empowering patients with knowledge about kidney health maintenance.",
                    "Workshops on diet, lifestyle, and disease management for renal patients.",
                    "Collaborations with healthcare professionals for comprehensive education."
                ]
            },
            "Dialysis access management": {
                "vendors": ["AccessPoint Care", "VascularAccess Solutions", "FlowGuard Management", "DialyAccess Clinics", "VeinCare Dialysis Services"],
                "comments": [
                    "Expert care in creating and maintaining dialysis access sites.",
                    "Minimally invasive procedures to maximize treatment efficacy.",
                    "Regular assessments to ensure access site health and function."
                ]
            },
            "Transplant coordination services": {
                "vendors": ["TransplantLink Coordination", "NewLife Transplant Services", "MatchOrgan Network", "HopeTransplant Connect", "LifeGift Transplant Support"],
                "comments": [
                    "Guidance through the kidney transplant process.",
                    "Support for patients on the transplant waiting list.",
                    "Collaboration with transplant centers for seamless patient care."
                ]
            },
            "Nutritional counseling for dialysis patients": {
                "vendors": ["NutriRenal Advisors", "DietWellness Renal", "KidneyNutri Care", "BalancedBites Counseling", "RenalDiet Solutions"],
                "comments": [
                    "Tailored dietary plans to support kidney health and dialysis treatment.",
                    "Professional advice on managing fluid and mineral intake.",
                    "Regular follow-ups to adjust dietary plans as needed."
                ]
            },
            "Psychological support services": {
                "vendors": ["MindRenal Support", "DialyzeMind Wellness", "RenalSpirit Counseling", "PsycheRenal Health", "EmbraceWellness Therapy"],
                "comments": [
                    "Emotional and psychological support tailored for renal patients.",
                    "Counseling services to help cope with the challenges of chronic kidney disease.",
                    "Group therapy sessions and peer support networks."
                ]
            },
            "Anemia management in dialysis": {
                "vendors": ["HemoBoost Therapies", "IronFlow Treatments", "AnemiaCare Dialysis Support", "BloodHealth Solutions", "VitalHem Solutions"],
                "comments": [
                    "Comprehensive anemia management protocols for dialysis patients.",
                    "Monitoring and treatment with iron supplements and EPO therapy.",
                    "Individualized care plans to address the root causes of anemia."
                ]
            },
            "Fluid management technology": {
                "vendors": ["HydraBalance Tech", "FluidWise Systems", "AquaControl Dialysis", "FlowRegulate Innovations", "Liquidus Management Devices"],
                "comments": [
                    "Advanced technologies for precise fluid removal during dialysis.",
                    "Customizable treatment settings to meet patient-specific needs.",
                    "Continuous innovation in fluid management for enhanced patient safety."
                ]
            },
            "Infection prevention and control": {
                "vendors": ["CleanDialysis Environments", "InfectoGuard Protocols", "SteriRenal Systems", "SafeDialyze Practices", "PathoShield Measures"],
                "comments": [
                    "Rigorous infection control measures to ensure patient safety.",
                    "Implementation of CDC guidelines in all treatment and common areas.",
                    "Regular training for staff on hygiene and infection prevention."
                ]
            }
        }
    },
    "Historical Sites": {
        "activities": {
            "Preservation and restoration projects": {
                "vendors": ["HeritagePreserve Constructors", "PastRenew Partners", "EraRestore Solutions", "TimeKeepers Restoration", "LegacyBuilders Co."],
                "comments": [
                    "Dedicated to maintaining the authenticity of historical structures.",
                    "Utilizing traditional materials and techniques for restoration.",
                    "Collaborating with historians to ensure historical accuracy."
                ]
            },
            "Educational programs and tours": {
                "vendors": ["HistoryWalks Education", "TimeTales Tours", "PastPaths Guides", "EduJourneys Historical", "HeritageExplorers Co."],
                "comments": [
                    "Engaging and informative tours for all age groups.",
                    "Customized educational programs for schools and universities.",
                    "Utilizing technology to enhance the learning experience."
                ]
            },
            "Exhibit design and installation": {
                "vendors": ["EraDisplays Solutions", "ExhibitPast Creators", "TimelineDesigns Co.", "HistoriCraft Installations", "ShowcaseHistory Partners"],
                "comments": [
                    "Creating immersive and interactive exhibits.",
                    "Highlighting significant historical events and figures.",
                    "Incorporating multimedia elements for dynamic presentations."
                ]
            },
            "Conservation research and studies": {
                "vendors": ["HeritageResearch Lab", "ConservaStudies Group", "AncientAnalytics Co.", "PastProbe Researchers", "TimeTested Science"],
                "comments": [
                    "Advanced research for preserving historical artifacts.",
                    "Developing new methods for long-term conservation.",
                    "Collaborating with academic institutions for comprehensive studies."
                ]
            },
            "Cultural heritage events": {
                "vendors": ["CulturaFest Organizers", "HeritageHappenings Events", "EpochEvents Planning", "AncestralCelebrations Co.", "TraditionsAlive LLC"],
                "comments": [
                    "Showcasing traditional crafts, music, and dances.",
                    "Promoting understanding and appreciation of cultural diversity.",
                    "Annual events to celebrate and preserve local heritage."
                ]
            },
            "Archaeological excavation support": {
                "vendors": ["DigDeep Archaeology", "PastLayers Excavations", "ArtifactFinders Co.", "HistoryUnearthed Services", "GroundStories LLC"],
                "comments": [
                    "Expert teams for sensitive archaeological digs.",
                    "Uncovering and documenting historical artifacts.",
                    "Working closely with historians and archaeologists for site preservation."
                ]
            },
            "Historical documentation and archiving": {
                "vendors": ["ArchiveMasters Services", "EraDocs Solutions", "PastRecords Co.", "HeritageFiles Organization", "TimeCapsule Archives"],
                "comments": [
                    "Digitizing and preserving historical documents.",
                    "Creating accessible archives for research and education.",
                    "Ensuring the longevity of valuable historical records."
                ]
            },
            "Landscape restoration and maintenance": {
                "vendors": ["HeritageGrounds Landscaping", "HistoricSites Gardening", "PastScapes Management", "EraGardens Care", "LegacyLawns Services"],
                "comments": [
                    "Restoring historical site landscapes to their original state.",
                    "Maintaining the natural beauty and historical integrity of sites.",
                    "Employing eco-friendly practices for sustainable site management."
                ]
            },
            "Historical site marketing and promotion": {
                "vendors": ["HistoryHighlights Marketing", "PastPromos Co.", "EraAttractions Advertising", "HeritageBuzz Solutions", "LegacyLure Campaigns"],
                "comments": [
                    "Raising awareness and interest in historical sites.",
                    "Developing engaging marketing campaigns for increased visitation.",
                    "Utilizing social media to reach a broader audience."
                ]
            },
            "Visitor services and amenities": {
                "vendors": ["TimeTraveler Amenities", "VisitorVault Services", "HeritageHosts Co.", "PastComforts Facilities", "HistoryHaven Concessions"],
                "comments": [
                    "Enhancing the visitor experience with quality amenities.",
                    "Providing informative and friendly services for guests.",
                    "Offering refreshments and merchandise inspired by historical themes."
                ]
            }
        }
    },
    "Bowling Centers": {
        "activities": {
            "League play organization": {
                "vendors": ["StrikeMasters Leagues", "PinChampions Coordinators", "BowlLeague Creators", "AlleyChamps Organizers", "KingpinLeagues Services"],
                "comments": [
                    "Hosting competitive leagues for all skill levels.",
                    "Creating a community atmosphere with regular play.",
                    "Organizing seasonal and themed bowling tournaments."
                ]
            },
            "Youth bowling programs": {
                "vendors": ["JuniorRollers Academy", "KidsBowl Club", "FutureStrikers Youth", "PinPals Junior Leagues", "LittleBowler Coaching"],
                "comments": [
                    "Introducing children to the sport in a fun, supportive environment.",
                    "Teaching fundamentals and sportsmanship through structured programs.",
                    "Hosting family-friendly events and youth tournaments."
                ]
            },
            "Bowling equipment sales and rental": {
                "vendors": ["ProShop Gear", "StrikeZone Equipment", "AlleyEquip Sales", "BowlEssentials Store", "PinGear Rentals"],
                "comments": [
                    "Offering a wide range of bowling balls, shoes, and accessories.",
                    "Providing expert advice for equipment selection.",
                    "Featuring the latest technology in bowling gear for optimal performance."
                ]
            },
            "Corporate and group events": {
                "vendors": ["GroupBowl Events", "CorporateStrikes Planners", "TeamPin Gatherings", "EventBowl Solutions", "StrikeTeam Organizing"],
                "comments": [
                    "Tailoring packages for team-building and corporate outings.",
                    "Offering private lanes and meeting spaces for groups.",
                    "Customizing events with catering and entertainment options."
                ]
            },
            "Bowling instruction and coaching": {
                "vendors": ["BowlCoach Institute", "StrikeSkill Trainers", "PinPros Lessons", "AlleyMasters Coaching", "FrameUp Instruction"],
                "comments": [
                    "Improving skills through personalized coaching sessions.",
                    "Catering to both beginners and advanced players.",
                    "Utilizing video analysis for technique improvement."
                ]
            },
            "Food and beverage services": {
                "vendors": ["LaneSnacks Cafe", "PinBites Grill", "BowlBar Beverages", "StrikeEats Restaurant", "GutterGourmet Catering"],
                "comments": [
                    "Enhancing the bowling experience with quality dining options.",
                    "Featuring a menu of favorite foods and craft beverages.",
                    "Hosting themed food nights and happy hour specials."
                ]
            },
            "Bowling alley maintenance and technology": {
                "vendors": ["PinSet Mechanics", "LaneTech Solutions", "FrameFix Services", "AlleyCare Maintenance", "BowlSys Tech"],
                "comments": [
                    "Ensuring optimal lane conditions with regular maintenance.",
                    "Incorporating the latest scoring and pinsetter technology.",
                    "Offering a seamless experience with well-maintained equipment."
                ]
            },
            "Special events and themed nights": {
                "vendors": ["ThemeBowl Nights", "AlleyFest Events", "CosmicBowl Parties", "RetroRoll Back", "GlowPin Evenings"],
                "comments": [
                    "Hosting unique themed nights for memorable experiences.",
                    "Attracting diverse crowds with costume, music, and decade nights.",
                    "Creating a vibrant and entertaining atmosphere for all ages."
                ]
            },
            "Membership and loyalty programs": {
                "vendors": ["BowlClub Memberships", "PinPerks Loyalty", "StrikeSavers Club", "FrameRewards Program", "AlleyAdvantage Benefits"],
                "comments": [
                    "Offering exclusive benefits and discounts to members.",
                    "Building a loyal community with rewards and recognition.",
                    "Providing special offers and early access to events."
                ]
            },
            "Arcade and additional entertainment": {
                "vendors": ["ArcadeStrike Zone", "PinPlay Arcade", "AlleyGames Entertainment", "Bowl&Play Centers", "GameFrame Arcade"],
                "comments": [
                    "Complementing bowling with a variety of arcade games.",
                    "Catering to families and younger guests with diverse entertainment.",
                    "Regularly updating game selections for fresh experiences."
                ]
            }
        }
    },
    "Political Organizations": {
        "activities": {
            "Voter education and registration drives": {
                "vendors": ["VoteReady Campaigns", "ElectAware Initiatives", "DemocracyBoost Org", "RegisterNow Networks", "CivicDuty Partners"],
                "comments": [
                    "Enhancing public awareness on the importance of voting.",
                    "Facilitating easy access to voter registration resources.",
                    "Organizing community workshops for informed electoral participation."
                ]
            },
            "Campaign strategy and management": {
                "vendors": ["StrategyPol Consultants", "CampaignEdge Solutions", "ElectionWin Advisors", "PolManage Experts", "VoteCraft Strategies"],
                "comments": [
                    "Developing comprehensive campaign strategies for candidates.",
                    "Providing end-to-end campaign management services.",
                    "Utilizing data analytics for targeted voter outreach."
                ]
            },
            "Political advocacy and lobbying": {
                "vendors": ["AdvocateVoice Group", "PolicyPush Lobbyists", "ChangeMakers Coalition", "ActionAgenda Advocates", "CivicInfluence Partners"],
                "comments": [
                    "Representing constituent interests at legislative levels.",
                    "Engaging in policy advocacy for social and political change.",
                    "Building coalitions for broader impact on public policy."
                ]
            },
            "Fundraising and donor management": {
                "vendors": ["FundFuture Campaigns", "DonateWell Services", "ElevateFunds Platform", "PledgeProspect Initiative", "SupporterSync Solutions"],
                "comments": [
                    "Implementing effective fundraising strategies for political causes.",
                    "Managing donor relations and recurring contribution programs.",
                    "Leveraging digital platforms for campaign financing."
                ]
            },
            "Social media and digital campaigning": {
                "vendors": ["DigitalVoice Campaigns", "SocialSphere Strategies", "NetInfluence Marketing", "OnlineElect Solutions", "WebCampaign Creators"],
                "comments": [
                    "Maximizing online presence for political campaigns.",
                    "Engaging voters through targeted social media content.",
                    "Analyzing digital outreach efforts for optimized engagement."
                ]
            },
            "Issue-based advocacy campaigns": {
                "vendors": ["CauseChampion Org", "IssueAdvocates Network", "RightFocus Campaigns", "UnityVoice Coalition", "ActionForChange Group"],
                "comments": [
                    "Focusing on key social, environmental, and economic issues.",
                    "Raising public awareness and support for legislative action.",
                    "Mobilizing grassroots movements for policy advocacy."
                ]
            },
            "Political event planning and coordination": {
                "vendors": ["EventPolitik Planners", "RallyReady Coordinators", "DebateStage Organizers", "TownHall Productions", "ElectEvents Company"],
                "comments": [
                    "Organizing rallies, debates, and town hall meetings.",
                    "Coordinating logistics for large-scale political events.",
                    "Creating impactful experiences for candidate support and advocacy."
                ]
            },
            "Public relations and media outreach": {
                "vendors": ["MediaMatters Relations", "PressPoint Communications", "PublicVoice PR", "CampaignSpotlight PR", "ElectorateEngage Media"],
                "comments": [
                    "Managing media relations to shape public perception.",
                    "Crafting press releases and managing news coverage.",
                    "Strategizing media outreach to elevate political profiles."
                ]
            },
            "Research and policy analysis": {
                "vendors": ["PolicyInsight Research", "ElectAnalytica Thinktank", "VoteMetrics Analysis", "CivicStudies Institute", "StrategySphere Analytics"],
                "comments": [
                    "Conducting research to inform policy positions and advocacy.",
                    "Analyzing electoral trends and voter behavior.",
                    "Providing strategic insights for campaign and policy development."
                ]
            },
            "Grassroots organizing and mobilization": {
                "vendors": ["GrassrootsEmpower Network", "MobilizeAction Group", "CommunityVoice Organizers", "RootsRise Movements", "CivicMobilize Collective"],
                "comments": [
                    "Building community networks for political engagement.",
                    "Empowering volunteers through training and resources.",
                    "Organizing door-to-door campaigns and local events for voter outreach."
                ]
            }
        }
    }
}



In [ ]:
import pandas as pd
import numpy as np

def generate_record(activity, vendor, cost, comment, naics_title, csv_columns):
    """
    Dynamically generate a record based on the provided parameters and csv_columns.

    Parameters:
    - activity: The business activity description.
    - vendor: The vendor name.
    - comment: The comment associated with the activity.
    - naics_title: The 2017 NAICS Title.
    - csv_columns: List of column names.
    - cost: The cost in USD.

    Returns:
    - A dictionary representing a single record.
    """
    # Initialize record with column names from csv_columns
    record = dict.fromkeys(csv_columns, "")

    # Dynamically assign values to keys based on their order in csv_columns
    record[csv_columns[0]] = activity
    record[csv_columns[1]] = vendor
    record[csv_columns[2]] = cost
    record[csv_columns[3]] = comment
    record[csv_columns[4]] = naics_title

    return record

def generate_dataset(naics_info, csv_columns):
    records = []
    for index in range(10):
      for naics_title, details in naics_info.items():
          for activity, info in details['activities'].items():
              for vendor in info['vendors']:
                  for comment in info['comments']:
                      # Generating a random cost for each record
                      cost_usd = np.random.random() * 1000
                      # Generate each record using the updated function
                      record = generate_record(activity, vendor, cost_usd, comment, naics_title, csv_columns)
                      records.append(record)
    return pd.DataFrame(records, columns=csv_columns)

# Assuming naics_info is your structured data for NAICS information
dataset = generate_dataset(naics_info, csv_columns)

# Split into training and test sets
train_dataset = dataset.sample(frac=0.833, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

# Save to CSV files
# Upload or update the training and test datasets
upload_dataframe(drive, folder_id, train_dataset, total_file_name)
upload_dataframe(drive, folder_id, train_dataset, train_file_name)
upload_dataframe(drive, folder_id, test_dataset, test_file_name)

# train_dataset.to_csv(file_folder+'sustainability_business_activities_training.csv', index=False)
# test_dataset.to_csv(file_folder+'sustainability_business_activities_test.csv', index=False)

print(f"Dataset generated with {len(train_dataset)} training records, {len(test_dataset)} test records for {len(naics_info.items())} titles.")


Created: sustainability_business_activities.csv
Updated: sustainability_business_activities_training.csv
Updated: sustainability_business_activities_test.csv
Dataset generated with 13744 training records, 2756 test records for 11 titles.
